In [1]:
import pandas as pd
import numpy as np
import math
from jellyfish import jaro_winkler
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from py_common_subseq import find_common_subsequences
import numbers
import time
from collections import Counter 
from fuzzywuzzy import fuzz
import re

In [2]:
#NOTES
#removing duplicate token-ID keys inplace caused some strange ID issue, so I am not removing duplicates anymore.  could be improved.
#unique_token_freq_max should really be derived dynamically by the number of tokens, some math
#jaro score threshold is too low.  Try correlation matrix to identify the best score to use, not sure it is jaro

In [3]:
import io
import shutil

with io.open('mint orgs.txt', encoding='utf-8', errors='ignore') as source:
    with io.open('mint orgs_utf.txt', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

In [20]:
right_df = pd.read_table('mint orgs_utf.txt',keep_default_na=False,error_bad_lines=False)
right_df.head()

Skipping line 943: expected 10 fields, saw 11
Skipping line 1897: expected 10 fields, saw 11
Skipping line 2024: expected 10 fields, saw 11
Skipping line 2379: expected 10 fields, saw 11
Skipping line 9308: expected 10 fields, saw 11
Skipping line 12943: expected 10 fields, saw 11
Skipping line 20599: expected 10 fields, saw 11
Skipping line 22142: expected 10 fields, saw 11
Skipping line 22434: expected 10 fields, saw 11
Skipping line 23297: expected 10 fields, saw 11
Skipping line 24235: expected 10 fields, saw 11
Skipping line 24955: expected 10 fields, saw 11
Skipping line 25932: expected 10 fields, saw 11
Skipping line 26535: expected 10 fields, saw 11
Skipping line 26581: expected 10 fields, saw 11
Skipping line 29338: expected 10 fields, saw 11
Skipping line 31345: expected 10 fields, saw 11
Skipping line 32432: expected 10 fields, saw 11
Skipping line 35982: expected 10 fields, saw 11
Skipping line 37408: expected 10 fields, saw 11
Skipping line 40387: expected 10 fields, saw 1

,rec_id,cbi_orgid,NAME,STREET1,STREET2,CITY,STPROV,ZIP,PHONE,URL
0,1,NULL,1ST ARMORED DIVISION ASSOCIATION,1815 LAKEWOOD DR,,ELIZABETHTOWN,KY,42702,502-737-0901,unknown
1,2,NULL,1ST CANADIAN PARACHUTE BATTALION ASSOCIATION,360 BLOOR ST E,APT 610,TORONTO,ON,M4W 3M3,416-927-0118,NULL
2,3,96749,1st Cavalry Division Association,302 North Main Street,,Copperas Cove,TX,76522,(254) 547-6537,http://www.1cda.org
3,4,83774,1st Fighter Group Association,11512 Henegan Pl.,NULL,Spotsylvania,VA,22551,308-632-3946,1stfighterassociation.weebly.com/
4,5,85835,1st Marine Division Association,1132 E. St.,Camp Pendleton,North,CA,92055,760-763-3268,1stmarinedivisionassociation.org


In [ ]:
overall_time = time.time()

#define column names
l_id = 'organization_id'
l_name = 'org_name'
l_address1 = 'address1'
l_city = 'city'
l_state = 'state'
l_zip = 'postal_code'
l_phone = 'phone'
l_web = 'website'
l_acronym = 'acronym'
l_alt_name = 'alt_name'

r_id = 'rec_id'
r_name = 'NAME'
r_address1 = 'STREET1'
r_city = 'CITY'
r_state = 'STPROV'
r_zip = 'ZIP'
r_phone = 'PHONE'
r_web = 'URL'
#r_acronym = 'CBI_Acronym'

#set parameters
token_match_min = 2 # minimum number of matched tokens to be considered a match
token_limiter = .995 # percent of non-single tokens to tokenize, where rare tokens are at the bottom and common at the top
unique_token_freq_max = 5 #threshold <= to a token is considered "unique" and links to these tokens are counted double
name_weight = .75 #note that this is really .75 * 4 because there are 4 org name simularity metrics
name_uniqueness_weight = 1.5
state_weight = 1
zip_weight = 1
phone_weight = 2
domain_weight = 2
name_score_min = 3
composite_score_min = 4 #minimum composite match score to be considered a match

start_time = time.time()
print "LOADING INITIAL DATAFRAMES..."

left_df = pd.read_csv('all cupola orgs w phone_db_utf.csv',keep_default_na=False)
right_df = pd.read_table('mint orgs_utf.txt',keep_default_na=False,error_bad_lines=False)

left_df.rename(columns={l_id:'id',l_name:'l_org_name',l_address1:'l_address1',l_city:'l_city',l_state:'l_state',l_zip:'l_postal_code',l_web:'l_web',l_phone:'l_phone',l_acronym:'l_acronym',l_alt_name:'l_alt_name'}, inplace=True)
right_df.rename(columns={r_id:'id',r_name:'r_org_name',r_address1:'r_address1',r_city:'r_city',r_state:'r_state',r_zip:'r_postal_code',r_web:'r_web',r_phone:'r_phone'}, inplace=True)

print("Dataframes loaded --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "PRE-PROCESSING..."
#normalize state codes
state_lkup = pd.read_csv('state_lkup.csv',keep_default_na=False)

from collections import defaultdict
state_dict = defaultdict(list)
for state, acronym in zip(state_lkup.state.values,state_lkup.acronym.values):
    state_dict[state].append(acronym)

left_df.l_state = left_df.l_state.str.lower()
left_df.l_state = left_df.l_state.replace(state_dict)
right_df.r_state = right_df.r_state.str.lower()
right_df.r_state = right_df.r_state.replace(state_dict)

#clean up non numeric characters in phones
l_clean_phones = []
for phone in left_df.l_phone:
    l_clean_phones.append(re.sub('[^0-9]','', phone))
    
left_df['l_clean_phone'] = l_clean_phones

r_clean_phones = []
for phone in right_df.r_phone:
    r_clean_phones.append(re.sub('[^0-9]','', phone))

right_df['r_clean_phone'] = r_clean_phones

#isolate domains from web URLs
l_domains = []
for web in left_df.l_web:
    l_domains.append(web.split('//')[-1].split('/')[0].strip('www.'))
left_df['l_domain'] = l_domains
    
r_domains = []
for web in right_df.r_web:
    r_domains.append(web.split('//')[-1].split('/')[0].strip('www.'))
right_df['r_domain'] = r_domains

print("states, phones, domains normalized --- %s seconds ---" % (time.time() - start_time))
print ""


start_time = time.time()
print "TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS..."

left_unique_token_columns = [ #tokens here circumvent the token_limiter which is cutting off the most commonly occuring tokens
    'l_state', 
    'l_postal_code',
    'l_clean_phone'

]

left_delta_token_columns = [
    'l_acronym',
    'l_org_name',
    'l_alt_name',
    #'l_address1',
    #'l_address2',
    'l_city',
    'l_domain'
]

right_unique_token_columns = [
    'r_state', 
    'r_postal_code',
    'r_clean_phone'
]

right_delta_token_columns = [
    #'r_acronym',
    'r_org_name',
    #'r_alt_name',
    #'r_address1',
    #'r_address2',
    'r_city',
    'r_domain'
]

# lowercase the name and split on spaces, remove non-alphanumeric chars
def tokenize_name(name):
    if isinstance(name, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in name)
        return clean_name.lower().split()
    else:
        return name
    
unique_tokens = []    
for col in left_unique_token_columns:
    for word in left_df[col]:
        if isinstance(word, float) is False:
            unique_tokens.append(tokenize_name(str(word)))
            
for col in right_unique_token_columns:
    for word in right_df[col]:
        if isinstance(word, float) is False:
            unique_tokens.append(tokenize_name(str(word)))
            
unique_flat_list = [item for sublist in unique_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
u_cnt = Counter()
for token in unique_flat_list:
    u_cnt[token] += 1

u_cnt_dict = dict(u_cnt) #convert to dictionary

unique_tokens_df = pd.DataFrame(u_cnt_dict.items(), columns=['token', 'count'])
unique_tokens_df = unique_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#consider waiting to do the count flag thing later, instead use some type of "token type" code
unique_token_flag = []
for index, value in enumerate(unique_tokens_df['count']):
    if value == 1:
        unique_token_flag.append(0)  #for any tokens occuring only once, we exclude
    else:
        unique_token_flag.append(1)

unique_tokens_df['flag'] = unique_token_flag        

all_other_words = []
for col in left_delta_token_columns:
    for word in left_df[col]:
        if isinstance(word, float) is False:
            all_other_words.append(tokenize_name(str(word)))
            
for col in right_delta_token_columns:
    for word in right_df[col]:
        if isinstance(word, float) is False:
            all_other_words.append(tokenize_name(str(word)))
            
flat_list = [item for sublist in all_other_words for item in sublist] #flatten list so it can be counted

#instantiate counter and use to count word frequencies in flat list
cnt = Counter()
for token in flat_list:
    cnt[token] += 1

cnt_dict = dict(cnt) #convert to dictionary

main_tokens_df = pd.DataFrame(cnt_dict.items(), columns=['token', 'count'])
main_tokens_df = main_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#wait to do count until joined with unique tokens?
main_token_flag = []
for index, value in enumerate(main_tokens_df['count']):
    if value == 1:
        main_token_flag.append(0)  #for any tokens occuring only once, we exclude
    elif index < int(main_tokens_df.shape[0] * token_limiter): #important line, we are cutting the top x% of frequently occuring tokens
        main_token_flag.append(1)
    else:
        main_token_flag.append(0)  #for the most common tokens, we exclude

main_tokens_df['flag'] = main_token_flag

all_tokens = pd.concat([unique_tokens_df, main_tokens_df])

all_tokens.drop('count',axis=1,inplace=True)
all_tokens['flag'] = all_tokens.flag.astype(int) #converting flags to int
tokens_dct = all_tokens.to_dict('split') #converting tokens_df to dictionary
tokens_dct=dict(tokens_dct['data']) #honestly can't remember why this works, something to do with conversion to dictionary

#preparing token_ids which will be used for joining left and right dfs
all_tokens.sort_values(by='flag',ascending=False,inplace=True)
all_tokens.sort_values(by='token',inplace=True)
all_tokens.drop_duplicates(subset='token',keep='first',inplace=True)
token_ids = all_tokens.index.get_level_values(0)
all_tokens['token_id'] = token_ids

all_tokens.drop('flag',axis=1,inplace=True)
all_tokens['token_id'] = all_tokens.token_id.astype(int)
token_id_dct = all_tokens.to_dict('split')
tokens_id_dct=dict(token_id_dct['data'])

vocabulary = np.array([w for w, c in tokens_dct.items() if c ==1]) #this works even without the ==1 and I don't know why
cv = CountVectorizer( vocabulary=vocabulary)

#now we are ready to tokenize left and right dataframes
all_left_cols = left_unique_token_columns + left_delta_token_columns

left_frame_list = []
for colname in all_left_cols:
    tokenmapping = cv.fit_transform(left_df[colname])
    df_row, token_id = tokenmapping.nonzero()

    left_frame_list.append(pd.DataFrame(np.vstack([vocabulary[token_id], left_df['id'].values[df_row]]).T, columns = ['token', 'id_l']))

left_keyed = pd.concat(left_frame_list)
left_keyed.drop_duplicates()#inplace=True)
#removing duplicates inplace was giving me a very strange issue where a small percentage of token_ids would be excluded from the left_keyed index

#append token_id to token as this will be more efficient to join with
left_token_ids = []
for token in left_keyed.token:
    left_token_ids.append(tokens_id_dct[token])

left_keyed['token_id'] = left_token_ids
left_keyed.sort_values(by='token_id',inplace=True)
left_keyed.set_index('token_id',inplace=True)
left_keyed.drop('token',axis=1,inplace=True)

all_right_cols = right_unique_token_columns + right_delta_token_columns

right_frame_list = []
for colname in all_right_cols:
    tokenmapping = cv.fit_transform(right_df[colname])
    df_row, token_id = tokenmapping.nonzero()

    right_frame_list.append(pd.DataFrame(np.vstack([vocabulary[token_id], right_df['id'].values[df_row]]).T, columns = ['token', 'id_r']))

right_keyed = pd.concat(right_frame_list)
right_keyed.drop_duplicates()#inplace=True)

#append token_id to token as this will be more efficient to join with
right_token_ids = []
for token in right_keyed.token:
    right_token_ids.append(tokens_id_dct[token])

right_keyed['token_id'] = right_token_ids
right_keyed.sort_values(by='token_id',inplace=True)
right_keyed.set_index('token_id',inplace=True)
right_keyed.drop('token',axis=1,inplace=True)

aggregations = {
    'id_l': 'count'
}

joined = left_keyed.join(right_keyed, how='inner',lsuffix='_l',rsuffix='_r')
#double-counting unique token matches
bonus_point_tokens = []
for token in main_tokens_df[(main_tokens_df['count'] > 1) & (main_tokens_df['count']<=unique_token_freq_max)].token:
    bonus_point_tokens.append(tokens_id_dct[token])

intersection_bonus_tokens = set(bonus_point_tokens).intersection(set(list(joined.index)))    
    
bonus_token_joins = []
for token_id in intersection_bonus_tokens:
    bonus_token_joins.append(joined.loc[token_id])
    
bonus_joins = pd.concat(bonus_token_joins)
bonus_joins_cols = bonus_joins[['id_l','id_r']].copy()
bonus_joins_cols.dropna(inplace=True)
joined_bonus = pd.concat([joined,bonus_joins_cols])

keys_grouped = joined_bonus.groupby(by=['id_l', 'id_r']).agg(aggregations)
keys_grouped.rename(columns={'id_l':'id_l count'}, inplace=True)
matched_records = keys_grouped[keys_grouped['id_l count'] >= token_match_min]
matched_records.reset_index(inplace=True)

print("match candidates identified --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
left_df.rename(columns={'id':'id_l'},inplace=True)
right_df.rename(columns={'id':'id_r'},inplace=True)

left_match_data = left_df[['id_l','l_org_name','l_city','l_state','l_postal_code','l_domain','l_clean_phone']].copy()
right_match_data = right_df[['id_r','r_org_name','r_city','r_state','r_postal_code','r_domain','r_clean_phone']].copy()

#making sure keys are str, results in blank df otherwise
left_match_data.id_l = left_match_data.id_l.astype('str')
right_match_data.id_r = right_match_data.id_r.astype('str')
matched_records.id_l = matched_records.id_l.astype('str')
matched_records.id_r = matched_records.id_r.astype('str')

#merging matched_records df with original record data for ease of review
l_conc = pd.merge(matched_records, left_match_data, on='id_l')
full_conc = pd.merge(l_conc, right_match_data, on='id_r')

print("original data concatenated with matches --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING ORG NAME SIMULARITY..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on edit distance of org names
def jaro_simularity(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return jaro_winkler(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '')
def fuzz_partial(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.partial_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_sort(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_sort_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_set(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_set_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)

full_conc['l_org_name'] = full_conc['l_org_name'].astype('str')
full_conc['r_org_name'] = full_conc['r_org_name'].astype('str')

jaro_time = time.time()
full_conc['jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_org_name, x.r_org_name), axis=1)
print("jaro scores done --- %s seconds ---" % (time.time() - jaro_time))

jaro_reduced = full_conc[full_conc.jaro_score > .25]

partial_time = time.time()
jaro_reduced['fuzz_partial_score'] = jaro_reduced.apply(lambda x: fuzz_partial(x.l_org_name, x.r_org_name), axis=1)
print("fuzz partial scores done --- %s seconds ---" % (time.time() - partial_time))
sort_time = time.time()
jaro_reduced['fuzz_sort_score'] = jaro_reduced.apply(lambda x: fuzz_sort(x.l_org_name, x.r_org_name), axis=1)
print("fuzz sort scores done --- %s seconds ---" % (time.time() - sort_time))
set_time = time.time()
jaro_reduced['fuzz_set_score'] = jaro_reduced.apply(lambda x: fuzz_set(x.l_org_name, x.r_org_name), axis=1)
print("fuzz set scores done --- %s seconds ---" % (time.time() - set_time))
print ""

print("name simularity scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING NAME SEQUENCE UNIQUENESS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

org_tokens = []    
for word in left_df['l_org_name']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))

for word in right_df['r_org_name']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))
            
org_flat_list = [item for sublist in org_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
org_cnt = Counter()
for token in org_flat_list:
    org_cnt[token] += 1

org_cnt_dict = dict(org_cnt) #convert to dictionary

def sequence_uniqueness(seq):
    return sum(1/org_cnt_dict[str.lower(t)]**0.5 for t in seq)

def name_similarity(a, b):
    if isinstance(a,basestring) is True and isinstance(b,basestring) is True:
        if len(a) > 0 and len(b) > 0:
            a_tokens = set(tokenize_name(a))
            b_tokens = set(tokenize_name(b))
            a_uniq = sequence_uniqueness(a_tokens)
            b_uniq = sequence_uniqueness(b_tokens)

            return sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
    else:
        return 0

jaro_reduced['uniq'] = jaro_reduced.apply(lambda x: name_similarity(x.l_org_name, x.r_org_name), axis=1)

print("name uniqueness scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR STATE CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def sanitize_state(state):
    if isinstance(state,basestring) is True:
        return ''.join(c for c in (state or '') if c in 'abcdefghijklmnopqrstuvwxyz')
    else:
        return ''
    
def state_match(state_a, state_b):
    sanitized_state_a = str(sanitize_state(state_a))
    sanitized_state_b = str(sanitize_state(state_b))

    # if the value is too short, means it's fubar
    if len(sanitized_state_a) < 2 or len(sanitized_state_b) < 2:
        return 0
    if state_a == state_b:
        return 1
    else:
        return 0    

jaro_reduced['state_match'] = jaro_reduced.apply(lambda x: state_match(x.l_state, x.r_state), axis=1)

print("state codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR POSTAL CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching postal code

def sanitize_postal(postal):
    if isinstance(postal, basestring) is True:
        return ''.join(c for c in (postal or '') if c in '1234567890')
    if isinstance(postal, float) is False:
        return postal

def postal_simularity(postal_a, postal_b):
    sanitized_postal_a = str(sanitize_postal(postal_a))
    sanitized_postal_b = str(sanitize_postal(postal_b))

    # if the number is too short, means it's fubar
    if len(sanitized_postal_a) < 5 or len(sanitized_postal_b) < 5:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(sanitized_postal_a, sanitized_postal_b))) / 5 >= 1:
        return 1
    else:
        return 0
    
jaro_reduced['zip_match'] = jaro_reduced.apply(lambda x: postal_simularity(x.l_postal_code, x.r_postal_code), axis=1)
    
print("postal codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR WEB DOMAIN MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def domain_match(domain_a, domain_b):
    if isinstance(domain_a, basestring) is True and isinstance(domain_b, basestring) is True:
        if domain_a == domain_b:
            return 1
        else:
            return 0

jaro_reduced['domain_match'] = jaro_reduced.apply(lambda x: domain_match(x.l_domain, x.r_domain), axis=1)

print("web domains checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR PHONE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching phone
def phone_simularity(phone_a, phone_b):

    if len(phone_a) < 10 or len(phone_b) < 10:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(phone_a, phone_b))) / 10 >= 1:
        return 1
    else:
        return 0
    
jaro_reduced['phone_match'] = jaro_reduced.apply(lambda x: phone_simularity(x.l_clean_phone, x.r_clean_phone), axis=1)
    
print("phones checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "DISTILLING STRONG ORG MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

jaro_reduced['overall_name_score'] = jaro_reduced.jaro_score * name_weight \
+ jaro_reduced.fuzz_partial_score * name_weight \
+ jaro_reduced.fuzz_sort_score * name_weight \
+ jaro_reduced.fuzz_set_score * name_weight \
+ jaro_reduced.uniq * name_uniqueness_weight

#calculate composite match score based on component scores and weights
jaro_reduced['composite_match_score'] = jaro_reduced.overall_name_score \
+ jaro_reduced.zip_match * zip_weight \
+ jaro_reduced.state_match * state_weight \
+ jaro_reduced.domain_match * domain_weight \
+ jaro_reduced.phone_match * phone_weight

#we take any matches meeting either name match threshold or composite match threshold as matches for review
org_matches = jaro_reduced[(jaro_reduced.overall_name_score >= name_score_min) | (jaro_reduced.composite_match_score >= composite_score_min)]

print("final matches isolated --- %s seconds ---" % (time.time() - start_time))
print ""

print("TOTAL COMPUTE TIME --- %s seconds ---" % (time.time() - overall_time))

org_matches.sort_values(by='composite_match_score', ascending=False)

LOADING INITIAL DATAFRAMES...


Skipping line 943: expected 10 fields, saw 11
Skipping line 1897: expected 10 fields, saw 11
Skipping line 2024: expected 10 fields, saw 11
Skipping line 2379: expected 10 fields, saw 11
Skipping line 9308: expected 10 fields, saw 11
Skipping line 12943: expected 10 fields, saw 11
Skipping line 20599: expected 10 fields, saw 11
Skipping line 22142: expected 10 fields, saw 11
Skipping line 22434: expected 10 fields, saw 11
Skipping line 23297: expected 10 fields, saw 11
Skipping line 24235: expected 10 fields, saw 11
Skipping line 24955: expected 10 fields, saw 11
Skipping line 25932: expected 10 fields, saw 11
Skipping line 26535: expected 10 fields, saw 11
Skipping line 26581: expected 10 fields, saw 11
Skipping line 29338: expected 10 fields, saw 11
Skipping line 31345: expected 10 fields, saw 11
Skipping line 32432: expected 10 fields, saw 11
Skipping line 35982: expected 10 fields, saw 11
Skipping line 37408: expected 10 fields, saw 11
Skipping line 40387: expected 10 fields, saw 1

Dataframes loaded --- 0.723000049591 seconds ---

PRE-PROCESSING...
states, phones, domains normalized --- 1.34100008011 seconds ---

TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS...
match candidates identified --- 267.397000074 seconds ---


C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


original data concatenated with matches --- 38.5650000572 seconds ---

SCORING ORG NAME SIMULARITY...
jaro scores done --- 1284.34299994 seconds ---


C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fuzz partial scores done --- 3155.148 seconds ---


In [15]:
len(full_conc.loc[276996].r_org_name)

0

In [ ]:
full_conc.head()

In [16]:
def sequence_uniqueness(seq):
    return sum(1/org_cnt_dict[str.lower(t)]**0.5 for t in seq)

def name_similarity(a, b):
    if isinstance(a,basestring) is True and isinstance(b,basestring) is True:
        if len(a) > 0 and len(b) > 0:
            a_tokens = set(tokenize_name(a))
            b_tokens = set(tokenize_name(b))
            a_uniq = sequence_uniqueness(a_tokens)
            b_uniq = sequence_uniqueness(b_tokens)

            return sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
    else:
        return 0

full_conc['uniq'] = full_conc.apply(lambda x: name_similarity(x.l_org_name, x.r_org_name), axis=1)